In this notebook, you can find the code used to detect recordings every 20 frames, in addition to saving them in a CSV file called history.

In [ ]:
import os
import cv2
import pandas as pd
from PIL import Image
from tqdm import tqdm_notebook
from project_utils.fo_dataset_pipeline import *

In [ ]:
# path to the definitive model of the previous work or the preliminary model of the extended work
model_path = '../faster_rcnn/preliminary_model/mmdet.faster_rcnn_resnext101_32x4d_fpn_1x.pth'

# loading the model
model_type, model, img_size, class_map = get_model_from_checkpoint(model_path)

# defining the list of transformatios to apply to the inference images
infer_tfms = get_tfms(img_size=img_size, tfms_type = 'valid')

# definig the GPU as the device to make the predictions
device = torch.device('cuda') 

# loading model to the GPU
model = model.to(device) 

In [ ]:
webcam_no = 1 # defining the webcam number (1 or 2)

webcam_path = f'/media/lucaszampar/BACKUP/webcam0{webcam_no}/'  # defining the path to the recordings
detected_videos_path = f'{webcam_path}detected_videos/' # defining path to move the detected recordings

history_path = f'{webcam_path}history/' # defining the path to save the detections in the history csv file
history_filename = 'history.csv'

In [ ]:
# if any of the above paths do not exist, they will be created as well as the history csv file containing the detections
if not os.path.exists(history_path):
    os.mkdir(history_path)
    history = pd.DataFrame(columns=['webcam', 'filename', 'frame_pos', 'labels', 'scores', 'bboxes'])
    history.to_csv(history_path+history_filename, index=False)
    
if not os.path.exists(detected_videos_path):
    os.mkdir(detected_videos_path)    

In [ ]:
frame_step = 20 # detections are performed every 20 frames

history = pd.read_csv(history_path+history_filename)
os.system(f'cp {history_path+history_filename} {history_path}history_bkp.csv') # The history csv file is backed up as a precaution

In [ ]:
# defining the recordings that will be detected for the day
files = [webcam_path+filename for filename in os.listdir(webcam_path) 
         if os.path.isfile(webcam_path+filename) and filename.split('/')[-1][0:8] in ['05062024']]

In [ ]:
#files

In [ ]:
#history

In [ ]:
for filepath in tqdm_notebook(files, desc=f'WEBCAM0{webcam_no}'):
    
    filename = filepath.split('/')[-1]

    video = cv2.VideoCapture(filepath)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    for frame_pos in tqdm_notebook(range(1, total_frames+1, frame_step), desc=f'Detectando {filename}: '):
        
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_pos-1)
        ret, frame = video.read()

        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_frame = PIL.Image.fromarray(frame)

            preds = model_type.end2end_detect(pil_frame, infer_tfms, model, class_map=class_map)
            bboxes = preds['detection']['bboxes']

            if len(bboxes) > 0: 
            
                labels = preds['detection']['labels']
                scores = preds['detection']['scores']

                new_history_row = pd.DataFrame({
                    'webcam':webcam_no,
                    'filename': filename,
                    'frame_pos': frame_pos-1, 
                    'frame_step': frame_step,
                    'labels': [labels], 
                    'scores': [scores], 
                    'bboxes': [[bbox.xyxy for bbox in bboxes]]
                })

                history = pd.concat([history, new_history_row], ignore_index=True)
    
    history.to_csv(f'{history_path}{history_filename}', index=False)
    os.system(f'mv {filepath} {detected_videos_path}{filename}')

In [ ]:
# saving the detections performed in the csv file
history = history.sort_values('filename').reset_index(drop=True)
history.to_csv(f'{history_path}{history_filename}', index=False)